In [8]:
import boa
import sys, os
from eth_utils import to_checksum_address
parent_dir = os.path.abspath('..')
sys.path.insert(0, parent_dir)
from tests.utils.util_tokens import mint_for_testing, get_token_balance
from tests.utils.util_contracts import load_contract


from dotenv import load_dotenv
dotenv_path = os.path.expanduser('~/.web3env') # Path to .env file containing rpcs and keys
load_dotenv(dotenv_path)

rpc_url = os.getenv("ETH_RPC_URL")
assert rpc_url is not None, "Provider url is not set"
boa.env.fork(url=rpc_url, block_identifier=20420069)
boa.env.enable_fast_mode()

addresses_dict = {
    "USDT":     '0xdAC17F958D2ee523a2206206994597C13D831ec7',
    "USDC":     '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    # "DAI":      '0x6b175474e89094c44da98b954eedeac495271d0f', #curve doesn't like DAI it seems
    "WBTC":     '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
    "WETH":     '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    "CRV":      '0xD533a949740bb3306d119CC777fa900bA034cd52',
    "crvUSD":   '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'
}
# Convert addresses to checksum format
for key in addresses_dict.keys():
    addresses_dict[key] = to_checksum_address(addresses_dict[key])
# Fetch contract data from Etherscan and create the contracts dictionary
# contracts = {key: boa.from_etherscan(address, uri="https://api.etherscan.io/api", name=key, api_key=os.environ.get("ETHERSCAN_API_KEY_CS")) for key, address in addresses_dict.items()}
token_contracts = {key: load_contract(address, name=key, path_prefix='../') for key, address in addresses_dict.items()}

print(f'Forked the chain on block {boa.env.evm.vm.state.block_number}')
boa.env.set_random_seed(420)
deployer = boa.env.generate_address()
with boa.env.prank(deployer):
    curve_assistant_contract = boa.load_partial("../contracts/CurveAssistant.vy").deploy()



Forked the chain on block 20420069


In [9]:
pools_list = ['0x7f86bf177dd4f3494b841a37e810a34dd56c829b', # tricrypto-ng USDC/WBTC/WETH
                '0xd51a44d3fae010294c616388b506acda1bfaae46', # Curve.fi: USDT/WBTC/WETH Pool (old)
                '0xf5f5b97624542d72a9e06f04804bf81baa15e2b4', # tricrypto-ng USDT/WBTC/WETH
                '0x4ebdf703948ddcea3b11f675b4d1fba9d2414a14', # tricrv (crvUSD/WETH/CRV)
                ] 
coins_list = []
coins_amounts = []

for contract in token_contracts.values():
    with boa.env.prank(deployer):
        coins_list.append(contract.address)
        mint_for_testing(contract, deployer, 10 ** contract.decimals(), path_prefix='../')
        coins_amounts.append(contract.balanceOf(deployer) // 2)
        print(f'{contract.name()} balance: {contract.balanceOf(deployer)/10 ** contract.decimals()}')
        tx = contract.approve(curve_assistant_contract.address, 0, sender = deployer)
        tx = contract.approve(curve_assistant_contract.address, contract.balanceOf(deployer), sender = deployer)
        print(f'{contract.name()} allowance: {contract.allowance(deployer, curve_assistant_contract.address)/10 ** contract.decimals()}')


Tether USD balance: 1.0
Tether USD allowance: 1.0
USD Coin balance: 1.0
USD Coin allowance: 1.0
Wrapped BTC balance: 1.0
Wrapped BTC allowance: 1.0
Wrapped Ether balance: 1.0
Wrapped Ether allowance: 1.0
Curve DAO Token balance: 1.0
Curve DAO Token allowance: 1.0
Curve.Fi USD Stablecoin balance: 1.0
Curve.Fi USD Stablecoin allowance: 1.0


In [10]:
with boa.env.prank(deployer):
    res_load = curve_assistant_contract.load_coins_to_pools(coins_list, coins_amounts, pools_list)
    print(f'returned value: {res_load}')


6263501305009555626
6046479956228384109
6242405581050548608
12821211143329243331


returned value: [6263501305009555626, 6046479956228384109, 6242405581050548608, 12821211143329243331]


In [11]:
### unload_liq  
pool_coins = []
for pool in pools_list:
    pool_contract = load_contract(pool, path_prefix='../')
    pool_coins.append([pool_contract.coins(i) for i in range(3)])
    lp_address = curve_assistant_contract.get_lp_token_for_pool(pool)
    lp_token_contract = load_contract(lp_address, path_prefix='../')
    print(f'Balance of {pool} LP token ({lp_token_contract.address}):', lp_token_contract.balanceOf(deployer))
    with boa.env.prank(deployer):
        lp_token_contract.approve(curve_assistant_contract.address, 0, sender = deployer)
        lp_token_contract.approve(curve_assistant_contract.address, lp_token_contract.balanceOf(deployer), sender = deployer)
res_unload = curve_assistant_contract.extract_liquidity(pools_list, sender = deployer)
print(f'returned value: {res_unload}')
# flatten and exclude duplicates from pool_coins
coins_list = list(set([item for sublist in pool_coins for item in sublist]))
for coin in coins_list:
    coin_contract = load_contract(coin, path_prefix='../')
    print(f'Balance of {coin_contract.name()} ({coin_contract.address}):', coin_contract.balanceOf(deployer)/10**coin_contract.decimals())


Balance of 0x7f86bf177dd4f3494b841a37e810a34dd56c829b LP token (0x7F86Bf177Dd4F3494b841a37e810A34dD56c829B): 6263501305009555626
Balance of 0xd51a44d3fae010294c616388b506acda1bfaae46 LP token (0xc4AD29ba4B3c580e6D59105FFf484999997675Ff): 6046479956228384109
Balance of 0xf5f5b97624542d72a9e06f04804bf81baa15e2b4 LP token (0xf5f5B97624542D72A9E06f04804Bf81baA15e2B4): 6242405581050548608
Balance of 0x4ebdf703948ddcea3b11f675b4d1fba9d2414a14 LP token (0x4eBdF703948ddCEA3B11f675B4D1Fba9d2414A14): 12821211143329243331
returned value: [True, True, True, True]
Balance of Wrapped BTC (0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599): 0.67361616
Balance of Wrapped Ether (0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2): 3.9921185611460666
Balance of USD Coin (0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48): 3788.501071
Balance of Curve DAO Token (0xD533a949740bb3306d119CC777fa900bA034cd52): 500.3604080654221
Balance of Tether USD (0xdAC17F958D2ee523a2206206994597C13D831ec7): 7572.659759
Balance of Curve.Fi USD

In [12]:
token_address = to_checksum_address('0x7f86bf177dd4f3494b841a37e810a34dd56c829b')
user_address = to_checksum_address('0x9970BE065e6751AC78Cd07C8E2b54F5D3B88c945')
source = \
f"""
from ethereum.ercs import IERC20

@external
@view
def get_balance(erc20_token_address: address, user_address: address) -> uint256:
    return staticcall IERC20(erc20_token_address).balanceOf(user_address)
"""
# print(source)
contract = boa.loads(source)
contract.get_balance(token_address, user_address)


0

In [13]:
from tests.utils.util_tokens import get_token_balance
with boa.env.prank(deployer):
    print(get_token_balance('0x7f86bf177dd4f3494b841a37e810a34dd56c829b') == get_token_balance('0x7f86bf177dd4f3494b841a37e810a34dd56c829b', deployer))

True


In [14]:
source = \
f"""
N_MAX: constant(uint256) = 10

@external
@view
def test():
    nest: DynArray[uint256, N_MAX] = []
    for i:uint256 in range(10):
        nest.append(i)
    print(5 in nest)
    print(nest)
"""
# print(source)
contract = boa.loads(source)
contract.test()


True
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
